In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18359542921969230793, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16932767575917202107
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 32
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 15

In [6]:
train_dir = 'data/train/'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.3,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 34909 images belonging to 6 classes.
Found 14958 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [8]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 34909
nb_validation_samples: 14958
nb_test_samples: 13578

predict_size_train: 1091
predict_size_validation: 468
predict_size_test: 425

 num_classes: 6


In [9]:
# vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
# nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
# nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
# mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"

In [10]:
# from keras.applications.vgg19 import VGG19
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
# from keras.applications.xception import Xception, preprocess_input
# from keras.applications import DenseNet201
# from keras.applications import DenseNet121
from keras.applications import ResNet50
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications import NASNetLarge, NASNetMobile
# from keras.applications import MobileNet

bottleneck_final_model=ResNet50(weights="imagenet", include_top=False, pooling = "avg")

In [11]:
# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "ResNet50_descriptors"

In [12]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [13]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [14]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [15]:

import tensorflow as tf
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     set_session(tf.Session(config=config))

reset_keras_tf_session()

In [16]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [17]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [18]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
  
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0)
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [19]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [20]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:  {'beta_1': 0.6, 'beta_2': 0.4, 'lr': 1e-05}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 2.0191 - acc: 0.1948 - val_loss: 1.6631 - val_acc: 0.2262
Epoch 2/15
 - 4s - loss: 1.8034 - acc: 0.2154 - val_loss: 1.6573 - val_acc: 0.2312
Epoch 3/15
 - 4s - loss: 1.7307 - acc: 0.2243 - val_loss: 1.6569 - val_acc: 0.2312
Epoch 4/15
 - 4s - loss: 1.6979 - acc: 0.2263 - val_loss: 1.6574 - val_acc: 0.2312
Epoch 5/15
 - 4s - loss: 1.6819 - acc: 0.2303 - val_loss: 1.6572 - val_acc: 0.2243
Epoch 00005: early stopping
14958/14958 [==============================] - 1s 36us/step
Validation Accuracy: 22.4295%
Validation Loss: 1.6572484461472847
Test Accuracy: 0.29164825452938575
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.4, 'beta_2': 0.3, 'lr': 0.01}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 4s

Test Accuracy: 0.3082191780821918
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.8, 'beta_2': 0.7, 'lr': 0.1}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 12.6144 - acc: 0.2166 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 2/15
 - 4s - loss: 12.6238 - acc: 0.2168 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 3/15
 - 4s - loss: 12.6238 - acc: 0.2168 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 4/15
 - 4s - loss: 12.6238 - acc: 0.2168 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 00004: early stopping
14958/14958 [==============================] - 1s 38us/step
Validation Accuracy: 21.6807%
Validation Loss: 12.623578551938024
Test Accuracy: 0.2154956547356017
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'

Epoch 4/15
 - 4s - loss: 1.6950 - acc: 0.2292 - val_loss: 1.6571 - val_acc: 0.2285
Epoch 00004: early stopping
14958/14958 [==============================] - 1s 39us/step
Validation Accuracy: 22.8506%
Validation Loss: 1.6571171910929512
Test Accuracy: 0.27758138164678153
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.4, 'beta_2': 0.999, 'lr': 1e-06}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 2.2363 - acc: 0.1460 - val_loss: 1.9121 - val_acc: 0.0823
Epoch 2/15
 - 4s - loss: 2.1193 - acc: 0.1634 - val_loss: 1.8200 - val_acc: 0.2264
Epoch 3/15
 - 4s - loss: 2.0298 - acc: 0.1772 - val_loss: 1.7603 - val_acc: 0.2264
Epoch 4/15
 - 4s - loss: 1.9700 - acc: 0.1905 - val_loss: 1.7222 - val_acc: 0.2264
Epoch 5/15
 - 4s - loss: 1.9296 - acc: 0.1964 - val_loss: 1.6985 - val_acc: 0.2264
Epoch 00005: early stopping
14958/14958 [====

Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 6s - loss: 1.7178 - acc: 0.2314 - val_loss: 1.6576 - val_acc: 0.2320
Epoch 2/15
 - 5s - loss: 1.6556 - acc: 0.2579 - val_loss: 1.6560 - val_acc: 0.1865
Epoch 3/15
 - 4s - loss: 1.6398 - acc: 0.2779 - val_loss: 1.6548 - val_acc: 0.2324
Epoch 4/15
 - 4s - loss: 1.6209 - acc: 0.3018 - val_loss: 1.6487 - val_acc: 0.1753
Epoch 5/15
 - 4s - loss: 1.6009 - acc: 0.3218 - val_loss: 1.6481 - val_acc: 0.1544
Epoch 6/15
 - 4s - loss: 1.5806 - acc: 0.3398 - val_loss: 1.6395 - val_acc: 0.1935
Epoch 00006: early stopping
14958/14958 [==============================] - 1s 41us/step
Validation Accuracy: 19.3542%
Validation Loss: 1.6394731566969414
Test Accuracy: 0.3443806157018707
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.9, 'beta_2': 0.995, 'lr': 0.001}
Train on 34909 samples, validate on 14958 samp

Epoch 4/15
 - 4s - loss: 1.5743 - acc: 0.3300 - val_loss: 1.6522 - val_acc: 0.1694
Epoch 00004: early stopping
14958/14958 [==============================] - 1s 44us/step
Validation Accuracy: 16.9408%
Validation Loss: 1.652202426279463
Test Accuracy: 0.3272941523051996
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.8, 'beta_2': 0.3, 'lr': 0.001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 6s - loss: 1.6811 - acc: 0.2329 - val_loss: 1.6616 - val_acc: 0.2264
Epoch 2/15
 - 4s - loss: 1.6481 - acc: 0.2505 - val_loss: 1.6552 - val_acc: 0.1496
Epoch 3/15
 - 4s - loss: 1.6362 - acc: 0.2563 - val_loss: 1.6535 - val_acc: 0.2193
Epoch 4/15
 - 4s - loss: 1.6301 - acc: 0.2566 - val_loss: 1.6510 - val_acc: 0.2454
Epoch 5/15
 - 4s - loss: 1.6370 - acc: 0.2499 - val_loss: 1.6568 - val_acc: 0.2312
Epoch 6/15
 - 4s - loss: 1.6568 - acc: 0.2307 - v

...

Epoch 6/15
 - 9s - loss: 1.4851 - acc: 0.3787 - val_loss: 1.6432 - val_acc: 0.1801
Epoch 7/15
 - 9s - loss: 1.4661 - acc: 0.3851 - val_loss: 1.6541 - val_acc: 0.2183
Epoch 8/15
 - 9s - loss: 1.4514 - acc: 0.3928 - val_loss: 1.6823 - val_acc: 0.2147
Epoch 9/15
 - 9s - loss: 1.4339 - acc: 0.4020 - val_loss: 1.6231 - val_acc: 0.2011
Epoch 10/15
 - 9s - loss: 1.4260 - acc: 0.4026 - val_loss: 1.6280 - val_acc: 0.1781
Epoch 00010: early stopping
14958/14958 [==============================] - 2s 143us/step
Validation Accuracy: 17.8099%
Validation Loss: 1.6279530957159887
Test Accuracy: 0.3631609957283842
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.5, 'beta_2': 0.9, 'lr': 1e-05}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 19s - loss: 1.8403 - acc: 0.2120 - val_loss: 1.6588 - val_acc: 0.2312
Epoch 2/15
 - 8s - loss: 1.7410 - acc: 0.2245

Parameters testing:  {'beta_1': 0.5, 'beta_2': 0.999, 'lr': 0.001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 20s - loss: 1.6740 - acc: 0.2371 - val_loss: 1.6544 - val_acc: 0.1659
Epoch 2/15
 - 10s - loss: 1.6289 - acc: 0.2824 - val_loss: 1.6590 - val_acc: 0.2374
Epoch 3/15
 - 10s - loss: 1.5802 - acc: 0.3257 - val_loss: 1.6451 - val_acc: 0.1765
Epoch 4/15
 - 10s - loss: 1.5402 - acc: 0.3525 - val_loss: 1.6543 - val_acc: 0.2047
Epoch 5/15
 - 9s - loss: 1.5076 - acc: 0.3650 - val_loss: 1.6686 - val_acc: 0.2387
Epoch 6/15
 - 10s - loss: 1.4850 - acc: 0.3758 - val_loss: 1.6556 - val_acc: 0.2063
Epoch 7/15
 - 11s - loss: 1.4632 - acc: 0.3861 - val_loss: 1.6568 - val_acc: 0.1946
Epoch 8/15
 - 10s - loss: 1.4559 - acc: 0.3893 - val_loss: 1.6729 - val_acc: 0.2189
Epoch 00008: early stopping
14958/14958 [==============================] - 2s 151us/step
Validation Accuracy: 21.8880%
Validation Loss: 1.6728560562072583
Test Accuracy: 0.36603328914420385
*_**_**_**_**_**_**_**

Epoch 2/15
 - 10s - loss: 1.6424 - acc: 0.2713 - val_loss: 1.6529 - val_acc: 0.2886
Epoch 3/15
 - 10s - loss: 1.6009 - acc: 0.3176 - val_loss: 1.6504 - val_acc: 0.1562
Epoch 4/15
 - 10s - loss: 1.5626 - acc: 0.3406 - val_loss: 1.6486 - val_acc: 0.1531
Epoch 5/15
 - 12s - loss: 1.5321 - acc: 0.3537 - val_loss: 1.6584 - val_acc: 0.1878
Epoch 00005: early stopping
14958/14958 [==============================] - 2s 154us/step
Validation Accuracy: 18.7792%
Validation Loss: 1.6583592593151868
Test Accuracy: 0.3373840035351304
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.4, 'beta_2': 0.7, 'lr': 1e-07}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 21s - loss: 2.0353 - acc: 0.1981 - val_loss: 1.7163 - val_acc: 0.2312
Epoch 2/15
 - 10s - loss: 2.0329 - acc: 0.1969 - val_loss: 1.7129 - val_acc: 0.2312
Epoch 3/15
 - 10s - loss: 2.0219 - acc: 0

Epoch 6/15
 - 10s - loss: 1.4807 - acc: 0.3760 - val_loss: 1.6651 - val_acc: 0.1956
Epoch 00006: early stopping
14958/14958 [==============================] - 2s 143us/step
Validation Accuracy: 19.5614%
Validation Loss: 1.6650531714828691
Test Accuracy: 0.35321844159670057
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.8, 'beta_2': 0.99, 'lr': 1e-05}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 22s - loss: 2.1859 - acc: 0.1624 - val_loss: 1.6692 - val_acc: 0.2264
Epoch 2/15
 - 11s - loss: 1.8088 - acc: 0.2146 - val_loss: 1.6575 - val_acc: 0.2312
Epoch 3/15
 - 11s - loss: 1.7344 - acc: 0.2215 - val_loss: 1.6572 - val_acc: 0.2127
Epoch 4/15
 - 11s - loss: 1.6988 - acc: 0.2280 - val_loss: 1.6575 - val_acc: 0.2312
Epoch 5/15
 - 11s - loss: 1.6821 - acc: 0.2313 - val_loss: 1.6580 - val_acc: 0.2275
Epoch 00005: early stopping
14958/14958

Epoch 2/15
 - 10s - loss: 1.6265 - acc: 0.2949 - val_loss: 1.6447 - val_acc: 0.2649
Epoch 3/15
 - 10s - loss: 1.5905 - acc: 0.3202 - val_loss: 1.6564 - val_acc: 0.1907
Epoch 4/15
 - 11s - loss: 1.5587 - acc: 0.3401 - val_loss: 1.6543 - val_acc: 0.2001
Epoch 5/15
 - 11s - loss: 1.5300 - acc: 0.3525 - val_loss: 1.6555 - val_acc: 0.1631
Epoch 00005: early stopping
14958/14958 [==============================] - 2s 139us/step
Validation Accuracy: 16.3123%
Validation Loss: 1.655502872521872
Test Accuracy: 0.3306819855648844
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
{'beta_1': 0, 'beta_2': 3, 'lr': 2}
